# Open Food Facts Data Exploration

In [1]:
import pandas as pd
import numpy as np

### Data Loading

In [2]:
data_folder = 'data/'
data = pd.read_csv(data_folder + 'en.openfoodfacts.org.products.csv', sep='\t')

/home/david/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,3,5,24,25,26,28,36,41,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We can see the size of the data is not a problem; we can load it easily using pandas.

### Data Preprocessing

#### Choosing The Fields
There are 174 different fields in this OpenFoodFacts dataset. After explorating each field and their value counts, here are the 33 fields we decided to keep (for now). The reasons we dropped the other fields are the following: either
- they were too poorly represented (less than 50 occurrences)
- they were too specific (micronutrients)
- some other field conveyed the same information in a clearer format
- they are of no interest for this study (barcodes, image urls)

In [3]:
chosen_fields = ['product_name', 'packaging_tags', 'brands_tags',
                 'origins_tags', 'manufacturing_places_tags', 'labels_en', 'stores', 'countries_en',
                 'additives_n', 'ingredients_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_n', 
                 'nutrition_grade_fr', 'pnns_groups_1', 'fruits-vegetables-nuts_100g',
                 'main_category_en', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 
                 'saturated-fat_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 
                 'omega-3-fat_100g', 'omega-6-fat_100g', 'omega-9-fat_100g', 'trans-fat_100g', 
                 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g', 'sodium_100g', 
                 'nutrition-score-fr_100g', 'nutrition-score-uk_100g']

In [4]:
chosen_data = data[chosen_fields]

#### Data Cleaning

##### Non-Numerical Fields

We are going to clean the packaging_tags, brands_tags, origins_tags, manufacturing_places_tags, labels_en, stores, countries_en, pnns_groups_1 fields in order to remove repetitions.

An easy example is pnnns_groups_1:

In [5]:
chosen_data['pnns_groups_1'].value_counts()

unknown                    122802
Sugary snacks               29975
Milk and dairy products     17932
Composite foods             14659
Cereals and potatoes        14575
Fish Meat Eggs              13970
Beverages                   12410
Fat and sauces              11400
Fruits and vegetables       11086
Salty snacks                 5591
fruits-and-vegetables        1537
sugary-snacks                1450
cereals-and-potatoes           25
salty-snacks                    3
Name: pnns_groups_1, dtype: int64

We can see that "Sugary snacks" (2nd position) and "sugary-snacks"(3rd to last position) should be the same.

##### Numerical Fields

The fields ending with "\_100g" and "\_n" are floats, we are going to investigate the values and eliminate the weird ones.

In [6]:
chosen_data.select_dtypes(include='float64').describe()

,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,fruits-vegetables-nuts_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,...,omega-6-fat_100g,omega-9-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,proteins_100g,sodium_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,394204.000000,394204.000000,394204.000000,3447.000000,5.950740e+05,912.000000,589691.000000,563782.000000,23748.000000,23788.000000,...,275.000000,32.000000,143944.000000,144627.000000,589436.000000,579046.000000,593642.000000,573874.000000,141399.000000,141399.000000
mean,1.573604,0.030753,0.070007,38.850017,1.177141e+03,562.548564,13.645716,5.355778,10.799654,6.482887,...,19.976836,35.469547,0.074164,0.023915,30.956099,15.768933,8.180591,1.052354,9.409543,8.825034
std,2.302502,0.174461,0.307981,34.693390,7.907707e+03,701.411498,111.995750,42.528905,18.916225,11.422893,...,54.628484,26.667426,1.544746,0.635228,272.673928,194.451028,63.648373,87.773859,8.887304,9.376735
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.005500,-3.570000,0.000000,0.000000,-17.860000,-800.000000,0.000000,-15.000000,-15.000000
25%,0.000000,0.000000,0.000000,2.500000,4.180000e+02,43.800000,0.500000,0.100000,0.210000,0.000000,...,1.635000,7.275000,0.000000,0.000000,3.900000,0.800000,1.000000,0.025988,2.000000,1.000000
50%,1.000000,0.000000,0.000000,35.000000,1.096000e+03,292.500000,6.300000,1.800000,4.100000,2.330000,...,8.900000,30.000000,0.000000,0.000000,16.670000,4.400000,5.400000,0.214000,10.000000,8.000000
75%,2.000000,0.000000,0.000000,60.450000,1.674000e+03,883.000000,21.000000,7.300000,10.710000,7.500000,...,24.000000,61.200000,0.000000,0.020000,55.000000,21.820000,11.000000,0.533000,16.000000,17.000000
max,30.000000,3.000000,6.000000,100.000000,2.845810e+06,3830.000000,66700.000000,18000.000000,1250.000000,417.000000,...,777.000000,75.000000,369.000000,127.000000,86700.000000,86700.000000,31000.000000,40000.000000,40.000000,40.000000


### Plan for Analysis and Communication

#### Step 1 - NutriScore

In [7]:
data_size = data.shape[0]
def calculate_apparition_pct(field):
    pct = (100*data[field].dropna().shape[0]/data_size)
    print('The apparition percentage is %.3f%%' % pct)
    return pct

Now that we have a cleaned dataset, we can start working on the analysis. 
The first step will be to calculate the [NutriScore](http://fr.openfoodfacts.org/score-nutritionnel-experimental-france) (we will keep the numerical value, instead of its corresponding alphabetical grade) where it is missing. 
We 

In [8]:
calculate_apparition_pct('nutrition_grade_fr');

The apparition percentage is 20.263%


We have all the parameters needed for the NutriScore formula (energy, saturated fats, sugars, proteins, sodium, fibers and "fruits, vegetables and nuts percentage" per 100g). 
Sadly, we can see below that the percentage of products where we could derive the NutriScore drops from 77.3% to 38.3% when we use the fiber. 
This might or might not be a problem, depending if the nans occur when a product doesn't have fibers (i.e. NaN corresponds to 0) or if they are simply missing. 
Thus, we will have to investigate.

A bigger problem however is the ridiculously small apparition percentage of the fruits-vegetables-nuts_100g field. Again, perhaps that the NaNs correspond to 0s. In any case, we will need to investigate. 

In [9]:
calculate_apparition_pct(['energy_100g', 'saturated-fat_100g', 'sugars_100g', 'proteins_100g', 'sodium_100g']);

The apparition percentage is 77.319%


In [10]:
calculate_apparition_pct(['energy_100g', 'saturated-fat_100g', 'sugars_100g', 'proteins_100g', 'sodium_100g', 'fiber_100g']);

The apparition percentage is 38.294%


In [11]:
calculate_apparition_pct('fruits-vegetables-nuts_100g');

The apparition percentage is 0.494%


One approach we consider is to use machine learning to calculate the NutriScore without the fruits-vegetables-nuts_100g.
In the worst case, where we don't find a smart way to tackle these issues, we will simply have overall smaller nutritional scores. 
Hence we will still be able to proceed with the analysis in any case. 

#### Step 2 - NutriScore Classification

Now that we have the NutriScore as benchmark, we can classify the different characteristics (brands, origins, manufacturing places, labels, stores and countries) and correlate it with the number of ingredients from palm oil and the number of additives.

For each of the aforementioned characteristics, we are going to rank their most represented values based on their respective NutriScore, using the mean (or another statistic like the median if we are not convinced by its usefullness). For instance, the 5 most represented brands are:

In [12]:
list(data['brands_tags'].value_counts()[:5].index)

['carrefour', 'auchan', 'u', 'casino', 'leader-price']

we wonder how they rank regarding to the NutriScore mean of their respective products.

#### Step 3 - Eco-Friendliness

Another classification that we consider is the packaging of the products. 
Indeed, after inspecting the packaging_tags and classifying them based on recyclability, we get another benchmark.
We can thus repeat step 2 and classify according to the eco-friendliness.

In [13]:
calculate_apparition_pct('packaging_tags')*data_size/100

The apparition percentage is 20.156%


140651.0

We can see the packaging_tags field has a considerable number of values (more than 140'000).

##### Travel Distance

Another evaluator to add to this "eco-friendliness" benchmark is the distance between the production and selling sites. 

In [14]:
calculate_apparition_pct(['packaging_tags', 'origins_tags', 'countries_en', 'brands'])*data_size/100

The apparition percentage is 5.512%


38467.00000000001

For instance the number of occurences for the brands is non-negligible (more than 38'000). Therefore we can benchmark the brands under this new "label".